In [1]:
import grizzly

# Load data from any source
df = grizzly.read_csv("data.csv")

# Perform manipulations
df_filtered = df.filter_eq("category", "electronics")
df_sorted = df_filtered.sort("price", ascending=True)

# Aggregate results
report = df_sorted.groupby_sum("brand", "sales")

# Export to your desired format
report.to_parquet("summary.parquet")

In [2]:
df.show()

+-------------+---------+-------+-------+
| category    | brand   | price | sales |
+-------------+---------+-------+-------+
| electronics | Apple   | 1000  | 50    |
| electronics | Samsung | 800   | 100   |
| electronics | Apple   | 1200  | 30    |
| home        | IKEA    | 100   | 200   |
| home        | IKEA    | 150   | 150   |
| electronics | Samsung | 700   | 80    |
+-------------+---------+-------+-------+
